# Bab 19: Training and Deploying TensorFlow Models at Scale

Bab ini akan membahas bagaimana melatih dan menerapkan model TensorFlow dalam skala besar. Kita akan belajar tentang deployment model ke TF Serving dan Google Cloud AI Platform, serta membahas penggunaan GPU dan strategi distribusi untuk mempercepat pelatihan.

## 1. Menyajikan Model TensorFlow (Serving a TensorFlow Model)

Setelah model TensorFlow dilatih, langkah selanjutnya adalah menerapkannya ke dalam produksi. Meskipun model dapat digunakan langsung dalam kode Python, seiring bertambahnya infrastruktur, lebih baik untuk membungkus model dalam layanan khusus yang hanya berfungsi untuk membuat prediksi. Hal ini akan memisahkan model dari sisa infrastruktur, memudahkan pembaruan versi model, scaling layanan, melakukan eksperimen A/B, dan memastikan semua komponen perangkat lunak mengandalkan versi model yang sama.

**TF Serving** adalah server model yang sangat efisien dan teruji, ditulis dalam C++. Ini dapat menangani beban tinggi, menyajikan berbagai versi model, dan secara otomatis menerapkan versi terbaru dari repositori model.

### 1.1 Mengekspor SavedModels

TensorFlow menyediakan fungsi `tf.saved_model.save()` untuk mengekspor model ke format **SavedModel**. Format ini menyimpan grafik komputasi model dan bobotnya. Ini juga dapat mencakup lapisan pra-pemrosesan sehingga model dapat langsung menerima data mentah di produksi.

Sebuah SavedModel mewakili versi model dan disimpan sebagai direktori yang berisi file `saved_model.pb` (mendefinisikan grafik komputasi sebagai protokol buffer serial) dan subdirektori `variables` (berisi nilai variabel). SavedModel juga dapat menyertakan subdirektori `assets` untuk data tambahan seperti file kosakata.

**SavedModel hanya dapat digunakan dengan model yang didasarkan secara eksklusif pada operasi TensorFlow**, mengecualikan `tf.py_function()` (yang membungkus kode Python arbitrer) dan model `tf.keras` dinamis.

In [1]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
import json
import requests
import grpc
!pip install tensorflow-serving-api
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

# Memuat dan menyiapkan data MNIST (seperti di Bab 10)
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_new = X_test[:3] # Contoh instance baru

# Membangun dan melatih model sederhana (seperti di Bab 10)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
              optimizer="sgd",
              metrics=["accuracy"])

# Gunakan subset kecil untuk pelatihan agar lebih cepat
history = model.fit(X_train_full[:5000], y_train_full[:5000], epochs=5)

# Mengekspor model ke format SavedModel
model_version = "0001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path)

print(f"Model disimpan di: {model_path}")

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4555 - loss: 1.7754
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6917 - loss: 0.9411
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7342 - loss: 0.7928
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7547 - loss: 0.6999
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.7931 - loss: 0.6252
Model disimpan di: my_mnist_model/0001


### 1.2 Menginstal TensorFlow Serving

Ada berbagai cara untuk menginstal TF Serving, termasuk menggunakan citra Docker. Menggunakan Docker sangat direkomendasikan karena mudah diinstal, tidak mengganggu sistem Anda, dan menawarkan kinerja tinggi.

**Langkah-langkah Instalasi TF Serving dengan Docker:**
1. **Instal Docker**: Pastikan Docker terinstal di sistem Anda.
2. **Pull Image TF Serving**: Unduh citra Docker resmi TF Serving:
   ```bash
   docker pull tensorflow/serving
   ```
3. **Jalankan Container TF Serving**: Jalankan perintah berikut dari direktori root proyek Anda (di mana model disimpan):
   ```bash
   docker run -it --rm -p 8500:8500 -p 8501:8501 \
     -v "$(pwd)/my_mnist_model:/models/my_mnist_model" \
     -e MODEL_NAME=my_mnist_model \
     tensorflow/serving
   ```
   - `-it`: Membuat container interaktif dan menampilkan output server.
   - `--rm`: Menghapus container saat dihentikan (untuk menghindari kekacauan).
   - `-p 8500:8500`: Meneruskan port TCP 8500 host ke port 8500 container (untuk gRPC API).
   - `-p 8501:8501`: Meneruskan port TCP 8501 host ke port 8501 container (untuk REST API).
   - `-v "$(pwd)/my_mnist_model:/models/my_mnist_model"`: Membuat direktori model di host tersedia di dalam container. Sesuaikan path jika berbeda.
   - `-e MODEL_NAME=my_mnist_model`: Mengatur variabel lingkungan `MODEL_NAME` di container, sehingga TF Serving tahu model mana yang akan disajikan. Secara default, ia akan mencari model di direktori `/models` dan secara otomatis menyajikan versi terbaru yang ditemukan.

Output dari perintah ini akan menunjukkan bahwa TF Serving telah berjalan dan memuat model Anda.

### 1.3 Melakukan Query TF Serving melalui REST API

REST API cocok untuk data input dan output yang tidak terlalu besar. Format JSON yang digunakan berbasis teks, yang efisien tetapi kurang efisien dibandingkan format biner untuk transfer data besar.

In [2]:
SERVER_URL_REST = 'http://localhost:8501/v1/models/my_mnist_model:predict'

input_data_json = json.dumps({
    "signature_name": "serving_default",
    "instances": X_new.tolist()
})

try:
    response_rest = requests.post(SERVER_URL_REST, data=input_data_json)
    response_rest.raise_for_status() # Menimbulkan exception jika ada error HTTP
    response_rest = response_rest.json()
    y_proba_rest = np.array(response_rest["predictions"])
    print("Prediksi dari REST API:")
    print(y_proba_rest.round(2))
except requests.exceptions.ConnectionError as e:
    print(f"Error koneksi ke TF Serving (REST): {e}")
    print("Pastikan TF Serving container berjalan dengan benar.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Error koneksi ke TF Serving (REST): HTTPConnectionPool(host='localhost', port=8501): Max retries exceeded with url: /v1/models/my_mnist_model:predict (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7a72eadc2c90>: Failed to establish a new connection: [Errno 111] Connection refused'))
Pastikan TF Serving container berjalan dengan benar.


### 1.4 Melakukan Query TF Serving melalui gRPC API

gRPC API lebih efisien untuk transfer data besar karena menggunakan format biner kompak dan protokol komunikasi efisien berbasis HTTP/2.

In [3]:
try:
    # Impor di sini agar tidak mengganggu jika tidak terinstal
    from tensorflow_serving.apis import predict_pb2
    from tensorflow_serving.apis import prediction_service_pb2_grpc

    channel = grpc.insecure_channel('localhost:8500')
    predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)

    request_grpc = predict_pb2.PredictRequest()
    request_grpc.model_spec.name = model_name
    request_grpc.model_spec.signature_name = "serving_default"

    # Mendapatkan nama input dari model Keras
    input_name_grpc = model.input_names[0]
    request_grpc.inputs[input_name_grpc].CopyFrom(
        tf.make_tensor_proto(X_new, dtype=tf.float32))

    response_grpc = predict_service.Predict(request_grpc, timeout=10.0)

    output_name_grpc = model.output_names[0]
    outputs_proto_grpc = response_grpc.outputs[output_name_grpc]
    y_proba_grpc = tf.make_ndarray(outputs_proto_grpc)

    print("\nPrediksi dari gRPC API:")
    print(y_proba_grpc.round(2))

except ImportError:
    print("\n tensorflow-serving-api atau grpcio tidak terinstal. Lewati pengujian gRPC.")
except grpc.RpcError as e:
    print(f"\nError gRPC ke TF Serving: {e}")
    print("Pastikan TF Serving container berjalan dengan benar dan gRPC port (8500) terbuka.")
except Exception as e:
    print(f"Terjadi kesalahan: {e}")

Terjadi kesalahan: 'Sequential' object has no attribute 'input_names'


### 1.5 Menerapkan Versi Model Baru

Secara berkala, TF Serving memeriksa versi model baru. Jika ditemukan, ia akan menangani transisi dengan baik: permintaan yang tertunda akan dijawab dengan versi model sebelumnya, sementara permintaan baru akan ditangani dengan versi baru. Setelah semua permintaan tertunda selesai, versi model sebelumnya akan dilepaskan.

Untuk menerapkan versi baru, cukup ekspor SavedModel ke subdirektori dengan nomor versi yang lebih tinggi.

```python
# Melatih model versi baru
model_new_version = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="tanh"), # Mengubah fungsi aktivasi untuk demonstrasi
    keras.layers.Dense(100, activation="tanh"),
    keras.layers.Dense(10, activation="softmax")
])
model_new_version.compile(loss="sparse_categorical_crossentropy",
                           optimizer="adam", # Mengubah optimizer
                           metrics=["accuracy"])
history_new = model_new_version.fit(X_train_full[:5000], y_train_full[:5000], epochs=5)

model_version_new = "0002" # Versi baru
model_path_new = os.path.join(model_name, model_version_new)
tf.saved_model.save(model_new_version, model_path_new)

print(f"Model versi baru disimpan di: {model_path_new}")
print("TF Serving akan secara otomatis memuat versi baru ini setelah beberapa saat.")
```

Secara default, TF Serving dapat mengaktifkan **batching otomatis** untuk permintaan yang diterima dalam waktu singkat, yang dapat meningkatkan kinerja secara signifikan dengan memanfaatkan kekuatan GPU.


In [4]:
# Melatih model versi baru
model_new_version = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="tanh"), # Mengubah fungsi aktivasi untuk demonstrasi
    keras.layers.Dense(100, activation="tanh"),
    keras.layers.Dense(10, activation="softmax")
])
model_new_version.compile(loss="sparse_categorical_crossentropy",
                           optimizer="adam", # Mengubah optimizer
                           metrics=["accuracy"])
history_new = model_new_version.fit(X_train_full[:5000], y_train_full[:5000], epochs=5)

model_version_new = "0002" # Versi baru
model_path_new = os.path.join(model_name, model_version_new)
tf.saved_model.save(model_new_version, model_path_new)

print(f"Model versi baru disimpan di: {model_path_new}")
print("TF Serving akan secara otomatis memuat versi baru ini setelah beberapa saat.")

Epoch 1/5


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


157/157 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.6502 - loss: 0.9928
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.8202 - loss: 0.4854
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8576 - loss: 0.3855
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8572 - loss: 0.3983
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - accuracy: 0.8686 - loss: 0.3616
Model versi baru disimpan di: my_mnist_model/0002
TF Serving akan secara otomatis memuat versi baru ini setelah beberapa saat.


## 2. Menerapkan Model ke Google Cloud AI Platform

**Google Cloud AI Platform** (sebelumnya dikenal sebagai Google Cloud ML Engine) adalah layanan cloud yang mengelola penyediaan dan konfigurasi VM GPU untuk pelatihan dan penerapan model TensorFlow.

### 2.1 Membuat Layanan Prediksi di GCP AI Platform

Langkah-langkah awal melibatkan pengaturan akun GCP, mengaktifkan API yang diperlukan (Storage API, AI Platform API), membuat bucket di Google Cloud Storage (GCS) untuk menyimpan model dan data, dan mengunggah model SavedModel ke GCS.

Setelah itu, model dan versi model perlu dikonfigurasi di AI Platform. Ini melibatkan penentuan nama model, versi, versi TensorFlow dan runtime, jenis mesin, dan jalur model di GCS.

In [5]:
 # Langkah-langkah ini dijalankan di Google Cloud Shell atau lingkungan dengan gcloud SDK terinstal

 # Pastikan Anda sudah login ke GCP dan memilih proyek yang benar
 # gcloud auth login
 # gcloud config set project your-gcp-project-id

 # 1. Buat bucket GCS (ganti 'your-unique-bucket-name' dengan nama bucket unik Anda)
 # gsutil mb gs://your-unique-bucket-name

 # 2. Unggah model SavedModel Anda ke GCS
 # gsutil cp -r my_mnist_model/ gs://your-unique-bucket-name/models/

 # 3. Buat model di AI Platform
 # gcloud ai-platform models create my_mnist_model \
 #    --regions global # Atau region spesifik seperti us-central1

 # 4. Buat versi model
 # gcloud ai-platform versions create v0001 \
 #    --model my_mnist_model \
 #    --origin gs://your-unique-bucket-name/models/my_mnist_model/0001/ \
 #    --runtime-version 2.11 \
 #    --framework TENSORFLOW \
 #    --python-version 3.9 \
 #    --machine-type n1-standard-4 # Contoh: Ubah sesuai kebutuhan dan kuota Anda

 # Catatan: Runtime version 2.11 dan Python version 3.9 adalah contoh versi terbaru saat ini.
 # Selalu periksa dokumentasi Google Cloud untuk versi yang didukung.

### 2.2 Menggunakan Layanan Prediksi

AI Platform menangani enkripsi dan autentikasi melalui SSL/TLS dan token. Direkomendasikan untuk menggunakan **service account** dengan hak akses terbatas untuk aplikasi klien Anda.

Untuk melakukan query layanan prediksi, Anda dapat menggunakan Google API Client Library.

In [6]:
# Contoh kode untuk menggunakan layanan prediksi (ini memerlukan service account key file)
# Pastikan Anda telah mengunduh file JSON kunci service account Anda dan menempatkannya di direktori yang benar.

# import os
# import googleapiclient.discovery

# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service_account_key.json"

# project_id = "your-gcp-project-id" # Ganti dengan ID proyek GCP Anda
# model_id = "my_mnist_model"
# model_path_gcp = f"projects/{project_id}/models/{model_id}"

# ml_resource = googleapiclient.discovery.build("ml", "v1").projects()

# def predict_gcp(X_data):
#     input_data_json_gcp = {"signature_name": "serving_default",
#                            "instances": X_data.tolist()}
#     request_gcp = ml_resource.predict(name=model_path_gcp, body=input_data_json_gcp)
#     response_gcp = request_gcp.execute()
#     if "error" in response_gcp:
#         raise RuntimeError(response_gcp["error"])
#     return np.array([pred[model.output_names[0]] for pred in response_gcp["predictions"]])

# try:
#     y_proba_gcp = predict_gcp(X_new)
#     print("\nPrediksi dari GCP AI Platform:")
#     print(y_proba_gcp.round(2))
# except Exception as e:
#     print(f"\nError saat memanggil GCP AI Platform: {e}")
#     print("Pastikan Anda telah mengaktifkan API, membuat model dan versi, dan kunci service account sudah benar.")

## 3. Menerapkan Model ke Perangkat Seluler atau Embedded

Untuk menerapkan model ke perangkat seluler atau embedded, model harus ringan dan efisien. **TFLite** adalah pustaka yang menyediakan alat untuk mengurangi ukuran model, mengurangi komputasi yang dibutuhkan, dan menyesuaikan model dengan kendala perangkat.

### 3.1 Mengurangi Ukuran Model

Konverter TFLite dapat mengambil SavedModel dan mengkompresnya ke format yang lebih ringan berdasarkan **FlatBuffers**. Ini adalah pustaka serialisasi lintas platform yang efisien yang dirancang untuk memuat langsung ke RAM tanpa pra-pemrosesan.

In [7]:
# Mengonversi SavedModel ke format TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(model_path) # Menggunakan model_path dari versi 0001
tflite_model = converter.convert()

with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)
print("Model berhasil dikonversi ke TFLite: converted_model.tflite")

Model berhasil dikonversi ke TFLite: converted_model.tflite


### 3.2 Kuantisasi (Quantization)

Kuantisasi adalah teknik untuk mengurangi ukuran model lebih lanjut dengan mengonversi bobot model ke bilangan bulat 8-bit, yang dapat mengurangi ukuran model hingga empat kali lipat dibandingkan float 32-bit.

**Post-training quantization**: Kuantisasi bobot setelah pelatihan selesai. Ini adalah pendekatan paling sederhana dan paling umum.
**Quantization-aware training**: Menambahkan operasi kuantisasi palsu ke model selama pelatihan agar model belajar untuk mengabaikan noise kuantisasi, sehingga bobot akhir lebih kuat terhadap kuantisasi.

In [8]:
# Contoh post-training quantization
converter_quant = tf.lite.TFLiteConverter.from_saved_model(model_path)
converter_quant.optimizations = [tf.lite.Optimize.DEFAULT] # Ini mencakup kuantisasi bobot default
tflite_model_quant = converter_quant.convert()

with open("converted_model_quant.tflite", "wb") as f:
    f.write(tflite_model_quant)
print("Model berhasil dikonversi dan dikuantisasi ke TFLite: converted_model_quant.tflite")

Model berhasil dikonversi dan dikuantisasi ke TFLite: converted_model_quant.tflite


## 4. Menggunakan GPU untuk Mempercepat Komputasi

Melatih jaringan saraf besar pada satu mesin dengan satu CPU dapat memakan waktu berhari-hari atau bahkan berminggu-minggu. Penggunaan **GPU (Graphics Processing Unit)** dapat mempercepat pelatihan secara signifikan, dari hari menjadi menit atau jam.

### 4.1 Mengelola RAM GPU

Secara default, TensorFlow secara otomatis mengambil semua RAM di semua GPU yang tersedia saat pertama kali komputasi dijalankan. Ini dilakukan untuk membatasi fragmentasi RAM GPU.

Beberapa cara untuk mengelola RAM GPU:
1. **Menetapkan GPU ke Proses Tunggal**: Mengatur variabel lingkungan `CUDA_VISIBLE_DEVICES` agar setiap proses hanya melihat kartu GPU tertentu.
2. **Membatasi Jumlah RAM**: Memberi tahu TensorFlow untuk hanya mengambil sejumlah RAM GPU tertentu.
3. **Pertumbuhan Memori (Memory Growth)**: Memberi tahu TensorFlow untuk mengambil memori hanya saat dibutuhkan (tidak akan pernah melepaskan memori yang sudah diambil).
4. **Virtual GPU**: Membagi satu GPU fisik menjadi dua atau lebih GPU virtual (atau logis).

In [9]:
# Mengelola pertumbuhan memori GPU (harus dijalankan di awal program)
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Memberi tahu TensorFlow untuk hanya mengalokasikan memori sesuai kebutuhan
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(f"{len(gpus)} Physical GPUs, {len(logical_gpus)} Logical GPUs")
    except RuntimeError as e:
        # Pertumbuhan memori harus diatur sebelum GPU diinisialisasi
        print(e)
else:
    print("Tidak ada GPU fisik yang ditemukan.")

Tidak ada GPU fisik yang ditemukan.


### 4.2 Menempatkan Operasi dan Variabel di Perangkat

Secara default, semua variabel dan operasi akan ditempatkan di GPU pertama (`/gpu:0`), kecuali untuk operasi dan variabel yang tidak memiliki kernel GPU (akan ditempatkan di CPU: `/cpu:0`).

Kita dapat secara manual menempatkan operasi dan variabel pada perangkat tertentu menggunakan konteks `tf.device()`.

In [10]:
a = tf.Variable(42.0)
print(f"Variabel 'a' ditempatkan di: {a.device}")

b = tf.Variable(42) # Integer variables often default to CPU
print(f"Variabel 'b' ditempatkan di: {b.device}")

with tf.device("/cpu:0"):
    c = tf.Variable(42.0)
print(f"Variabel 'c' (dipaksa ke CPU) ditempatkan di: {c.device}")

Variabel 'a' ditempatkan di: /job:localhost/replica:0/task:0/device:CPU:0
Variabel 'b' ditempatkan di: /job:localhost/replica:0/task:0/device:CPU:0
Variabel 'c' (dipaksa ke CPU) ditempatkan di: /job:localhost/replica:0/task:0/device:CPU:0


### 4.3 Eksekusi Paralel di Beberapa Perangkat

TensorFlow mengeksekusi operasi dalam grafik komputasi secara paralel saat memungkinkan. Ketika `tf.function` dijalankan, ia menganalisis grafiknya untuk menemukan operasi yang perlu dievaluasi dan menjalankan operasi dengan dependensi nol (operasi sumber) di antrean evaluasi perangkat masing-masing.

## 5. Melatih Model dalam Skala Besar Menggunakan Distribution Strategies API

TensorFlow menyediakan **Distribution Strategies API** yang menyederhanakan pelatihan model dalam skala besar, baik di beberapa GPU pada satu mesin maupun di beberapa server.

### 5.1 Paralelisme Data (Data Parallelism)

Paralelisme data adalah pendekatan di mana model direplikasi di setiap perangkat, dan setiap langkah pelatihan dijalankan secara bersamaan di semua replika, menggunakan mini-batch yang berbeda untuk setiap replika. Gradien yang dihitung oleh setiap replika kemudian dirata-ratakan dan hasilnya digunakan untuk memperbarui parameter model.

Dua pendekatan utama:
1. **Strategi Mirrored (Mirrored Strategy)**: Mencerminkan semua parameter model di semua GPU dan selalu menerapkan pembaruan parameter yang sama persis di setiap GPU. Ini sangat efisien, terutama pada satu mesin, menggunakan algoritma AllReduce.
2. **Strategi dengan Parameter Terpusat (Centralized Parameters)**: Menyimpan parameter model di luar perangkat GPU (worker), misalnya di CPU atau server parameter. Ini dapat menggunakan pembaruan sinkron atau asinkron.

**Pembaruan Sinkron (Synchronous Updates)**: Agregator menunggu semua gradien tersedia sebelum menghitung rata-rata gradien dan meneruskannya ke optimizer.
**Pembaruan Asinkron (Asynchronous Updates)**: Setiap replika memperbarui parameter model segera setelah selesai menghitung gradien.

In [11]:
# Contoh MirroredStrategy (untuk banyak GPU di satu mesin)
# Jika Anda tidak memiliki GPU, ini akan menggunakan CPU dan mungkin tidak menunjukkan manfaat paralelisme.
print("\n--- Menggunakan MirroredStrategy ---")
try:
    strategy = tf.distribute.MirroredStrategy()
    print(f"Jumlah perangkat yang terdeteksi oleh MirroredStrategy: {strategy.num_replicas_in_sync}")

    with strategy.scope():
        mirrored_model = keras.models.Sequential([
            keras.layers.Flatten(input_shape=[28, 28]),
            keras.layers.Dense(300, activation="relu"),
            keras.layers.Dense(100, activation="relu"),
            keras.layers.Dense(10, activation="softmax")
        ])
        mirrored_model.compile(loss="sparse_categorical_crossentropy",
                               optimizer="sgd",
                               metrics=["accuracy"])

    # Batch size harus dapat dibagi dengan jumlah replika
    batch_size_mirrored = 100 * strategy.num_replicas_in_sync
    print(f"Batch size untuk MirroredStrategy: {batch_size_mirrored}")
    history_mirrored = mirrored_model.fit(
        X_train_full[:batch_size_mirrored*5], y_train_full[:batch_size_mirrored*5],
        epochs=2, batch_size=batch_size_mirrored)
    print("Pelatihan dengan MirroredStrategy selesai.")

except ValueError as e:
    print(f"Error saat menginisialisasi MirroredStrategy: {e}")
    print("Pastikan Anda memiliki setidaknya satu GPU yang tersedia atau sesuaikan konfigurasi GPU Anda.")
except Exception as e:
    print(f"Terjadi kesalahan lain: {e}")


--- Menggunakan MirroredStrategy ---
Jumlah perangkat yang terdeteksi oleh MirroredStrategy: 1
Batch size untuk MirroredStrategy: 100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.1421 - loss: 2.2973 
Epoch 2/2
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 74ms/step - accuracy: 0.2765 - loss: 2.1754
Pelatihan dengan MirroredStrategy selesai.


### 5.2 Paralelisme Model (Model Parallelism)

Paralelisme model adalah pendekatan di mana model dipecah menjadi beberapa bagian dan setiap bagian dijalankan pada perangkat yang berbeda. Ini lebih rumit daripada paralelisme data dan keuntungan kinerja sangat bergantung pada arsitektur jaringan saraf.

## 6. Menjalankan Pekerjaan Pelatihan Besar di Google Cloud AI Platform

Untuk pekerjaan pelatihan skala besar, Google Cloud AI Platform dapat mengelola penyediaan dan konfigurasi VM GPU sesuai permintaan Anda. Anda menggunakan alat baris perintah `gcloud` untuk mengirimkan pekerjaan pelatihan.

In [12]:
# Contoh perintah untuk mengirimkan pekerjaan pelatihan ke AI Platform
# (Ini adalah perintah bash, tidak dijalankan di Jupyter secara langsung)

# gcloud ai-platform jobs submit training my_training_job_$(date +%Y%m%d_%H%M%S) \
#    --region us-central1 \
#    --scale-tier BASIC_GPU \
#    --runtime-version 2.11 \
#    --python-version 3.9 \
#    --package-path ./trainer \
#    --module-name trainer.task \
#    --staging-bucket gs://your-unique-bucket-name/staging \
#    --job-dir gs://your-unique-bucket-name/output \
#    -- \
#    --num-epochs 10

# Catatan:
# - Ganti 'your-unique-bucket-name' dengan nama bucket GCS Anda.
# - 'BASIC_GPU' adalah tier skala yang menyediakan satu worker dengan GPU. Untuk lebih banyak GPU/server, gunakan tier lain seperti 'STANDARD_GPU' atau 'PREMIUM_GPU'.
# - Folder 'trainer' dan file 'trainer/task.py' harus ada di direktori lokal Anda dan berisi kode pelatihan TensorFlow.
# - Pastikan Anda telah menginstal gcloud SDK dan terautentikasi.

### 6.1 Black Box Hyperparameter Tuning di AI Platform

AI Platform menyediakan layanan penyetelan hyperparameter **Bayesian optimization** yang kuat yang disebut **Google Vizier**. Ini dapat mengeksplorasi ruang hyperparameter secara efisien untuk menemukan kombinasi terbaik.

In [13]:
# Contoh file tuning.yaml untuk hyperparameter tuning
 # trainingInput:
 #   hyperparameters:
 #     goal: MAXIMIZE
 #     hyperparameterMetricTag: accuracy
 #     maxTrials: 10
 #     maxParallelTrials: 2
 #     params:
 #     - parameterName: learning_rate
 #       type: DOUBLE
 #       minValue: 0.001
 #       maxValue: 0.1
 #       scaleType: UNIT_LOG_SCALE
 #     - parameterName: num_neurons
 #       type: INTEGER
 #       minValue: 50
 #       maxValue: 300
 #       scaleType: UNIT_LINEAR_SCALE

Pekerjaan pelatihan Anda perlu menulis ringkasan (TensorBoard logs) untuk metrik yang ditentukan, yang akan dipantau oleh AI Platform untuk memandu proses penyetelan.

## Kesimpulan

Bab ini telah membahas berbagai teknik untuk melatih dan menerapkan model TensorFlow dalam skala besar. Kita telah melihat bagaimana TF Serving memungkinkan penerapan model yang efisien dan berskala, serta bagaimana Google Cloud AI Platform menyederhanakan pelatihan dan penerapan di cloud, termasuk penyetelan hyperparameter. Pemahaman tentang strategi distribusi dan pengelolaan GPU sangat penting untuk mengoptimalkan kinerja dalam skala besar.